In [ ]:
!pip install pandas sentence_transformers scikit-learn datasets transformers

In [ ]:
from huggingface_hub import login
login()  # This will prompt you to enter your Hugging Face API key


In [ ]:
import pandas as pd
# import numpy as np

In [ ]:
thedata=pd.read_csv(r"/kaggle/input/anime-sentence-similarity-csv/thedata2.csv")
# thedata=pd.read_csv(r"/kaggle/input/anime-recom-100/thedata100.csv")

In [ ]:
# sentences = [
#     "suggest me a action and adventure anime with pirate",
#     "suggest me a romance anime",
#     "suggest me a anime with adventure",
#     "toradora is a comedy anime with highschool romance"
# ]


In [ ]:

# sentences = [
#     "suggest me a action and adventure anime",
#     "Attack in titan is a anime where humans fight against titans",
#     "One piece is a anime about a pirate named Monkey D. Luffy",
#     "toradora is a romance anime"
# ]
# embeddings = model.encode(sentences)

# similarities = model.similarity(embeddings, embeddings)
# print(similarities)
# [3, 3]

In [ ]:
# thedata.isna().sum()

In [ ]:


# Initialize the model
model = SentenceTransformer('all-MiniLM-L6-v2')


In [ ]:
from sklearn.model_selection import train_test_split
from sentence_transformers.losses import CosineSimilarityLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers import InputExample


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from sklearn.model_selection import train_test_split
from sentence_transformers.readers import InputExample
from sentence_transformers.losses import CosineSimilarityLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction
from transformers import TrainingArguments
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from datasets import Dataset






# Split the data into train, validation, and test sets
train_data, temp_data = train_test_split(thedata, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)




# # Prepare the datasets for training
# train_examples = [
#     InputExample(texts=[row['description'], row['genre']], label=float(row['score'])) 
#     for _, row in train_data.iterrows()
# ]
# val_examples = [
#     InputExample(texts=[row['description'], row['genre']], label=float(row['score'])) 
#     for _, row in val_data.iterrows()
# ]
# test_examples = [
#     InputExample(texts=[row['description'], row['genre']], label=float(row['score'])) 
#     for _, row in test_data.iterrows()
# ]



train_dataset = Dataset.from_dict({
    "description": list(train_data["description"]),
    "genre" :list(train_data["genre"]),
    "label" :list(train_data["score"])
})


val_dataset = Dataset.from_dict({
    "description": list(val_data["description"]),
    "genre": list(val_data["genre"]),
    "label": list(val_data["score"])
})

test_dataset = Dataset.from_dict({
    "description": list(test_data["description"]),
    "genre": list(test_data["genre"]),
    "label": list(test_data["score"])
})



# Load a pretrained SentenceTransformer model
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Define a loss function
loss = CosineSimilarityLoss(model=model)

# Create evaluators for validation and test datasets
val_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_dataset["description"],
    sentences2=val_dataset["genre"],
    scores=val_dataset["label"],
    main_similarity=SimilarityFunction.COSINE,
    name="anime-recommendation-dev"
)

test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=test_dataset["description"],
    sentences2=test_dataset["genre"],
    scores=test_dataset["label"],
    main_similarity=SimilarityFunction.COSINE,
    name="anime-recommendation-test"
)

# Training arguments
training_args = SentenceTransformerTrainingArguments(
    output_dir="models/anime-recommendation",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    eval_strategy="steps",
    eval_steps=100,
    save_steps=100,
    logging_steps=1,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    save_total_limit=2,
    fp16=True
)

# Define a Trainer


trainer = SentenceTransformerTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    loss=loss,
    evaluator=val_evaluator
)

# Train the model
trainer.train()

# Evaluate the model on the test set
test_evaluator(model)

# Save the trained model
model.save("models/anime_recom/final")

# (Optional) Push the model to the Hugging Face Hub
model.push_to_hub("anime-recommendation-model")


In [ ]:
print(model)

In [ ]:
sentences = [
    "one piece",
    "Attack in titan ",
    "suggest me a romance anime",
    "toradora"
]
embeddings = model.encode(sentences)
# model.predict("suggest me a action and adventure anime with pirate")

similarities = model.similarity(embeddings, embeddings)
print(similarities)